In [46]:
from pytorch_lightning import seed_everything

seed_everything(42)

from physioex.data import PhysioExDataModule
from physioex.models import load_pretrained_model
from physioex.models.load import get_models_table
from physioex.train.networks.utils.target_transform import get_mid_label

import pandas as pd
import numpy as np

import pytorch_lightning as pl

Seed set to 42


In [48]:
model = load_pretrained_model(
    name="seqsleepnet",
    in_channels=3,
)
data_folder = "/mnt/guido-data/"
data_module = PhysioExDataModule(
    datasets=["hmc"],
    batch_size=256,
    preprocessing="xsleepnet",
    selected_channels=["EEG", "EOG", "EMG"],
    target_transform=None,
    data_folder=data_folder,
)

trainer = pl.Trainer(deterministic=True)

results = trainer.test(model, datamodule=data_module)[0]


/home/guido/github/physioex-private/physioex/models/load.py:55: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  ckpt_path = ckpt_path[check_table["sequence_length"] == sequence_length]
/home/guido/github/physioex-private/physioex/models/load.py:56: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  ckpt_path = ckpt_path[check_table["in_channels"] == in_channels]
/home/guido/github/physioex-private/physioex/models/load.py:57: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  ckpt_path = ckpt_path[check_table["loss"] == loss]
/home/guido/miniconda3/envs/physioex/lib/python3.12/site-packages/pytorch_lightning/utilities/parsing.py:208: Attribute 'nn' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['nn'])`.
2024-09-19 09:09:16.077 | INFO     | physioex.utils.data_folder:set_data_folder:26 - Data folder set to 

Testing: |                                                                                                    …

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.6565186977386475     │
│          test_ck          │    0.5421931147575378     │
│          test_f1          │    0.6361228823661804     │
│         test_loss         │    1.0590749979019165     │
│         test_macc         │    0.6081961393356323     │
│         test_mf1          │    0.5910953283309937     │
│          test_pr          │     0.675656795501709     │
│          test_rc          │    0.6565186977386475     │
└───────────────────────────┴───────────────────────────┘

In [51]:
results_df = pd.read_csv("papers/msd/results.csv")

results["dataset"] = "hmc"
results["k"] = 0

results_df = pd.concat([results_df, pd.DataFrame([results])])
results_df.to_csv("papers/msd/results.csv", index=False)

In [45]:

df = []
models_names = ["chambon2018", "seqsleepnet", "tinysleepnet"]
for model_name in models_names:
    model = load_pretrained_model(
        name=model_name
    )
    
    # print the number of parameters
    n_params = sum(p.numel() for p in model.parameters())
    # print the number of XX.K parameters or M parameters
    print(f"Model {model_name} has K parameters: {n_params/1000:.2f}K")
    print(f"Model {model_name} has M parameters: {n_params/1000000:.2f}M")

/home/guido/github/physioex-private/physioex/models/load.py:55: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  ckpt_path = ckpt_path[check_table["sequence_length"] == sequence_length]
/home/guido/github/physioex-private/physioex/models/load.py:56: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  ckpt_path = ckpt_path[check_table["in_channels"] == in_channels]
/home/guido/github/physioex-private/physioex/models/load.py:57: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  ckpt_path = ckpt_path[check_table["loss"] == loss]
/home/guido/miniconda3/envs/physioex/lib/python3.12/site-packages/pytorch_lightning/utilities/parsing.py:208: Attribute 'nn' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['nn'])`.
/home/guido/github/physioex-private/physioex/models/load.py:55: UserWarning: Boolean Series key will be 

1
Model chambon2018 has K parameters: 17.90K
Model chambon2018 has M parameters: 0.02M
Model seqsleepnet has K parameters: 137.73K
Model seqsleepnet has M parameters: 0.14M
Model tinysleepnet has K parameters: 1516.42K
Model tinysleepnet has M parameters: 1.52M


/home/guido/miniconda3/envs/physioex/lib/python3.12/site-packages/pytorch_lightning/utilities/parsing.py:208: Attribute 'nn' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['nn'])`.


In [33]:
results = pd.read_csv("papers/physioex/results_train_test_raw.csv")

results = results.round(3)

# convert the float columns to % format
float_col = ["test_acc", "test_f1", "test_ck", "test_pr", "test_rc"]
for col in float_col:
    results[col] = results[col].apply(lambda x: f"{x*100:.1f}\%")

# rename the columns
results = results.rename(
    columns={
        "model": "Model",
        "in_channels": "Channels",
        "dataset": "Dataset",
        "sequence_length": "Sequence Length",
        "test_acc": "Accuracy",
        "test_f1": "F1 Score",
        "test_ck": "Cohen's Kappa",
        "test_pr": "Precision",
        "test_rc": "Recall",
    }
)

# remove the other columns
results = results[
    [
        "Train/Test",
        "Model",
        "Channels",
        "Dataset",
        "Accuracy",
        "F1 Score",
        "Cohen's Kappa",
        "Precision",
        "Recall",
    ]
]

results.to_csv("papers/physioex/results_train_test.csv")

results.set_index(
    [
        "Train/Test",
        "Model",
        "Channels",
    ],
    inplace=True,
)
# convert it to latex file
# results.to_latex("papers/physioex/results_train_test.tex")

<>:16: SyntaxWarning: invalid escape sequence '\%'
<>:16: SyntaxWarning: invalid escape sequence '\%'
/tmp/ipykernel_3309058/843730700.py:16: SyntaxWarning: invalid escape sequence '\%'
  results[col] = results[col].apply(lambda x: f"{x*100:.1f}\%")
